In [1]:
import numpy as np
import pandas as pd

In [2]:
class simplex():
    def __init__(self, A, b, c):
        self.A = A
        self.b = b
        self.c = c
        self.X, self.NormG, self.funValue = [], [], []
    # 定义二次函数
    def fun(self, x):
        return (x.T)@self.A@x/2 + self.b@x + self.c
    
    def fun_list(self, array):
        funList = []
        for k in array[:,]:
            funList.append(self.fun(k))
        return funList
    
    # 梯度
    def objGradient(self, x):
        g = self.A @ np.array(x) + self.b
        gNorm = np.linalg.norm(g, axis = 0)
        return g, gNorm
    
    # 终止条件1
    def absSum(self, funList, XL):
        funList = np.array(funList) - np.array(self.fun(XL))
        return sum(funList**2)
    
    # 终止条件2
    def relativeDiff(self, XL, XH):
        return abs((self.fun(XH)-self.fun(XL))/self.fun(XL))
    
    # 得到一个单纯性
    def get_simplex(self, initial_x):
        funList = self.fun_list(initial_x)
        XL = initial_x[funList.index(sorted(funList)[0]), :]
        XH = initial_x[funList.index(sorted(funList)[-1]), :]
        XG = initial_x[funList.index(sorted(funList)[-2]), :]
        xn_1 = (initial_x.sum(axis = 0) - XH)
        xn_2 = 2*xn_1 - XH
        return XL, XH, XG, xn_1, xn_2, funList
    
    # 迭代主体
    def iterate(self, initial_x, epsilon1, epsilon2, alpha, beta):
        '''
        params:
        epsilon:迭代终止界限
        initial_x:初始的n+1个点，以array形式传入
        '''
        self.X, self.NormG, self.funValue = [], [], []
        XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
        self.X.append(XH.copy())
        g, gNorm = self.objGradient(XH)
        self.NormG.append(gNorm)
        self.funValue.append(self.fun(XH))
        x = initial_x
        while self.absSum(funList, XL) > epsilon1 and self.relativeDiff(XH, XL) > epsilon2:
            if self.fun(xn_2) < self.fun(XL):
                xn_3 = xn_1 + alpha(xn_2 - xn_1)
                if self.fun(xn_3) < self.fun(xn_2):
                    XH = xn_3
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XH))
            elif self.fun(XL)<=self.fun(xn_2)<self.fun(XG):
                XH = xn_2
                self.X.append(XH)
                g, gNorm = self.objGradient(XH)
                self.NormG.append(gNorm)
                self.funValue.append(self.fun(XH))
            elif self.fun(XG)<=self.fun(xn_2)<self.fun(XH):
                xn_4 = xn_1 + beta*(xn_2 - xn_1)
                XH = xn_4
                self.X.append(XH)
                g, gNorm = self.objGradient(XH)
                self.NormG.append(gNorm)
                self.funValue.append(self.fun(XH))
            elif self.fun(xn_2)>=self.fun(XH):
                xn_5 = xn_1 + beta*(XH - xn_1)
                if self.fun(xn_5) >= self.fun(XH):
                    for k in range(len(x)):
                        x[k,:] = XL + (x[k,:] - XL)/2
                    XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(x)
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XH))
                else:
                    XH = xn_5
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XH))
        return self.X, self.NormG, self.funValue

In [3]:
A = np.array([[2, 0], [0, 8]])
b = np.array([0, 0])
c = 0
epsilon = 0.01
eighth = simplex(A, b, c)

In [4]:
x = np.array([[1, 1], [0, 1], [1, -1]])
X, NormG, funValue = eighth.iterate(initial_x=x, epsilon1=0.01, epsilon2=0.01, alpha=1.2, beta=0.5)

In [5]:
res8 = pd.DataFrame({'迭代XH值':X, '梯度范数':NormG, '函数值':funValue})

In [6]:
res8

,迭代XH值,梯度范数,函数值
0,"[1, 1]",8.246211,5.000000
1,"[1.0, 0.5]",4.472136,2.000000
2,"[1.0, 0.25]",2.828427,1.250000
3,"[1.0, 0.125]",2.236068,1.062500
4,"[1.0, 0.0625]",2.061553,1.015625
5,"[1.0, 0.03125]",2.015564,1.003906
6,"[1.0, 0.015625]",2.003902,1.000977
7,"[1.0, 0.0078125]",2.000976,1.000244
8,"[1.0, 0.00390625]",2.000244,1.000061
9,"[1.0, 0.001953125]",2.000061,1.000015
